In [2]:
import streamlit as st
st.__version__

'1.39.0'

In [1]:
import os
import json
import re
import mmap
import pickle
from typing import List, Tuple
from functools import cache
from pathlib import Path
from dataclasses import dataclass
from sklearn.metrics.pairwise import cosine_similarity

from loguru import logger
from diskcache import Cache

from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain.schema import HumanMessage, Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from bert_score import score  # Импортируем BERTScore

from colorama import Fore, Back, Style
import random

import asyncio
from fastapi import FastAPI, UploadFile, File, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, validator, constr
from typing import List, Dict, Optional, Set
import logging
from logging.handlers import RotatingFileHandler
import aiofiles
from pathlib import Path
import hashlib
import mimetypes
from datetime import datetime, timedelta
import json
from functools import lru_cache
import redis
import pypdf
from pptx import Presentation
from docx import Document
from io import BytesIO
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pymorphy2
from natasha import (
    Segmenter, MorphVocab, NewsEmbedding, 
    NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, Doc
)
from PIL import Image
import pytesseract
import os
import tempfile
import aiohttp
from concurrent.futures import ProcessPoolExecutor
import re
import time

In [2]:
from pathlib import Path
from typing import List
import os
import hashlib
import json
import re
from datetime import datetime
import logging
from functools import lru_cache

import pandas as pd
import pypdf
from docx import Document as DocxDocument
from pptx import Presentation
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from dataclasses import dataclass

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
from bert_score import score  # Импортируем BERTScore

# Функция для оценки релевантности на основе BERTScore
def evaluate_relevance(query, retrieved_doc, generated_answer):
    P, R, F1 = score([generated_answer], [retrieved_doc], lang="ru")
    return F1.mean().item()

def evaluate_relevance_score(query, doc_text):
    """Вычисляем BERTScore между запросом и текстом документа"""
    _, _, F1 = score([query], [doc_text], lang="ru")
    return F1.mean().item()

In [4]:
from pdf2image import convert_from_path
import pytesseract
import pypdf

In [3]:
import os
from pathlib import Path
import mmap
import hashlib
from typing import List, Set, Dict
from functools import lru_cache
import logging
import json
from docx import Document as DocxDocument
import pandas as pd
from pptx import Presentation
import pypdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import re

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class Config:
    """Конфигурация для обработки документов"""
    chunk_size: int = 512
    chunk_overlap: int = 200
    cache_dir: str = "./cache"
    supported_extensions: set = frozenset({'.docx', '.xlsx', '.pptx', '.pdf', '.csv', '.md', '.json' , '.txt'})

class DocumentLoader:
    """Document loader with recursive directory traversal and caching"""
    
    def __init__(self, config: Config):
        self.config = config
        self.text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n", ". ", " ", ""],  # Приоритетные разделители
            chunk_size=config.chunk_size,
            chunk_overlap=config.chunk_overlap,
            length_function=len,
            keep_separator=False
        )
        self._path_hash_cache = {}
        
        # Создаем директорию для кэша если её нет
        os.makedirs(config.cache_dir, exist_ok=True)
        
    def _calculate_path_hash(self, path: str) -> str:
        """Вычисляет хэш структуры директории и содержимого файлов"""
        hasher = hashlib.sha256()
        
        for root, dirs, files in os.walk(path):
            dirs.sort()
            files.sort()
            
            hasher.update(root.encode('utf-8'))
            
            for file in files:
                file_path = os.path.join(root, file)
                if Path(file_path).suffix.lower() in self.config.supported_extensions:
                    stats = os.stat(file_path)
                    file_info = f"{file_path}|{stats.st_mtime}|{stats.st_size}"
                    hasher.update(file_info.encode('utf-8'))
                    
        return hasher.hexdigest()

    def _get_all_files(self, path: str) -> List[str]:
        """Рекурсивно получает все поддерживаемые файлы из директории"""
        files = []
        for root, _, filenames in os.walk(path):
            for filename in filenames:
                file_path = os.path.join(root, filename)
                if Path(file_path).suffix.lower() in self.config.supported_extensions:
                    files.append(file_path)
        return sorted(files)
    
    def preprocess_text(self, text: str) -> str:
        """Предварительная обработка текста перед разбиением на чанки"""
        if not text:
            return ""
            
        # Заменяем множественные переносы строк на одинарный
        text = re.sub(r'\n\s*\n+', '\n', text)
        
        # Убираем пробелы в начале и конце строк
        text = '\n'.join(line.strip() for line in text.split('\n'))
        
        # Убираем пустые строки в начале и конце текста
        text = text.strip()
        
        # Заменяем множественные пробелы на одинарные внутри строк
        text = re.sub(r' +', ' ', text)
        
        return text

    def _extract_text_from_docx(self, file_path: str) -> str:
        """Извлекает текст из DOCX файла"""
        doc = DocxDocument(file_path)
        return "\n".join([paragraph.text for paragraph in doc.paragraphs])

    def _extract_text_from_xlsx(self, file_path: str) -> str:
        """Извлекает текст из XLSX файла"""
        df = pd.read_excel(file_path)
        return df.to_string()

    def _extract_text_from_pptx(self, file_path: str) -> str:
        """Извлекает текст из PPTX файла"""
        prs = Presentation(file_path)
        text = []
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    text.append(shape.text)
        return "\n".join(text)

    # def _extract_text_from_pdf(self, file_path: str) -> str:
    #     """Извлекает текст из PDF файла"""
    #     text = []
    #     with open(file_path, 'rb') as file:
    #         pdf_reader = pypdf.PdfReader(file)
    #         for page in pdf_reader.pages:
    #             extracted_text = page.extract_text()
    #             if extracted_text:
    #                 text.append(extracted_text)
    #                 # text.append(f"Page {page+1}: {extracted_text}") # ****
    #     return "\n".join(text)

    def _extract_text_from_pdf(self, file_path: str) -> str:
        """Extracts text from PDF file, using OCR for image-based pages if necessary"""
        text = []
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = pypdf.PdfReader(file)
                for page_num, page in enumerate(pdf_reader.pages):
                    # Attempt text extraction
                    extracted_text = page.extract_text()
                    
                    # If no text found, use OCR
                    if not extracted_text:
                        images = convert_from_path(file_path, first_page=page_num + 1, last_page=page_num + 1)
                        for image in images:
                            extracted_text = pytesseract.image_to_string(image)
                    
                    # Add page label if you want to track pages in the output
                    # if extracted_text:
                    #     text.append(f"Page {page_num + 1}:\n{extracted_text}")
                        
        except Exception as e:
            logger.error(f"Error extracting text from PDF {file_path}: {str(e)}")
        
        return "\n".join(text)

    def _extract_text_from_csv(self, file_path: str) -> str:
        """Извлекает текст из CSV файла"""
        df = pd.read_csv(file_path)
        return df.to_string()

    def _read_text_file(self, file_path: str) -> str:
        """Читает текст из MD файла"""
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()

    def _read_json_file(self, file_path: str) -> str:
        """Читает текст из JSON файла"""
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return json.dumps(data, ensure_ascii=False, indent=2)

    @lru_cache(maxsize=1000)
    def _load_document_cached(self, file_path: str, path_hash: str) -> str:
        """Кэшированная загрузка документа с поддержкой различных форматов"""
        logger.debug(f"Loading document: {file_path}")
        try:
            extension = Path(file_path).suffix.lower()
            
            # Проверяем кэш
            cache_file = Path(self.config.cache_dir) / f"{hashlib.sha256(file_path.encode()).hexdigest()}.txt"
            if cache_file.exists():
                with open(cache_file, 'r', encoding='utf-8') as f:
                    return f.read()
            
            # Извлекаем текст в зависимости от формата
            if extension == '.docx':
                text = self._extract_text_from_docx(file_path)
            elif extension == '.xlsx':
                text = self._extract_text_from_xlsx(file_path)
            elif extension == '.pptx':
                text = self._extract_text_from_pptx(file_path)
            elif extension == '.pdf':
                text = self._extract_text_from_pdf(file_path)
            elif extension == '.csv':
                text = self._extract_text_from_csv(file_path)
            elif extension == '.md':
                text = self._read_text_file(file_path)
            elif extension == '.json':
                text = self._read_json_file(file_path)
            else:
                logger.warning(f"Unsupported file extension: {extension}")
                return ""
            
            # Сохраняем в кэш
            with open(cache_file, 'w', encoding='utf-8') as f:
                f.write(text)
                
            return text
                
        except Exception as e:
            logger.error(f"Error loading {file_path}: {str(e)}")
            return ""

    def create_documents(self, path: str) -> List[Document]:
        """Создает Langchain документы с метаданными из всех файлов в директории"""
        path_hash = self._calculate_path_hash(path)
        file_paths = self._get_all_files(path)
        logger.info(f"Found {len(file_paths)} supported files in {path}")
        
        documents = []
        for file_path in file_paths:
            # Загружаем текст
            text = self._load_document_cached(file_path, path_hash)
            
            # Предварительная обработка
            text = self.preprocess_text(text)
            
            if not text:
                continue
            
            # Получаем метаданные
            metadata = {
                "source": file_path,
                "filename": Path(file_path).name,
                "extension": Path(file_path).suffix.lower(),
                "relative_path": os.path.relpath(file_path, path),
                "created_date": datetime.fromtimestamp(os.path.getctime(file_path)).isoformat(),
                "modified_date": datetime.fromtimestamp(os.path.getmtime(file_path)).isoformat()
            }
            
            # Разбиваем на чанки
            texts = self.text_splitter.split_text(text)
            
            # Обрабатываем чанки
            processed_chunks = []
            for chunk in texts:
                chunk = chunk.strip()
                if chunk and len(chunk) <= self.config.chunk_size:
                    processed_chunks.append(chunk)
            
            # Создаем документы
            docs = [Document(page_content=chunk, metadata=metadata) for chunk in processed_chunks]
            documents.extend(docs)
            
            logger.debug(f"Created {len(docs)} chunks from {file_path}")
            
        return documents

    def process_documents(self, path: str) -> List[Document]:
        """Обрабатывает все документы в директории"""
        logger.info(f"Processing documents in {path}")
        return self.create_documents(path)

def analyze_chunks(documents: List[Document]):
    """Анализирует полученные чанки"""
    total_chunks = len(documents)
    total_chars = sum(len(doc.page_content) for doc in documents)
    avg_chunk_size = total_chars / total_chunks if total_chunks > 0 else 0
    
    print(f"\nAnalysis Results:")
    print(f"Total chunks: {total_chunks}")
    print(f"Average chunk size: {avg_chunk_size:.2f} characters")
    
    print("\nSample chunks:")
    for i, doc in enumerate(documents[:3]):
        # print(f"\nChunk {i+1}:")
        print(f"Length: {len(doc.page_content)}")
        print(f"Source: {doc.metadata['source']}")
        print(f"Content:\n{doc.page_content}")
        print("-" * 80)

NameError: name 'dataclass' is not defined

In [2]:
# Создаем конфигурацию с настройками по умолчанию
config = Config()
# Инициализируем загрузчик
loader = DocumentLoader(config)
# Обрабатываем документы
documents = loader.process_documents("../data/TRAIN/")

# Анализируем результаты
analyze_chunks(documents)

INFO:__main__:Processing documents in ../data/TRAIN/
INFO:__main__:Found 70 supported files in ../data/TRAIN/
INFO:__main__:Using OCR for page 1 in ../data/TRAIN/0.pdf
ERROR:__main__:Error performing OCR: tesseract is not installed or it's not in your PATH. See README file for more information.
INFO:__main__:Using OCR for page 5 in ../data/TRAIN/0.pdf
ERROR:__main__:Error performing OCR: tesseract is not installed or it's not in your PATH. See README file for more information.
INFO:__main__:Using OCR for page 8 in ../data/TRAIN/0.pdf
ERROR:__main__:Error performing OCR: tesseract is not installed or it's not in your PATH. See README file for more information.
INFO:__main__:Using OCR for page 10 in ../data/TRAIN/0.pdf
ERROR:__main__:Error performing OCR: tesseract is not installed or it's not in your PATH. See README file for more information.
INFO:__main__:Using OCR for page 12 in ../data/TRAIN/0.pdf
ERROR:__main__:Error performing OCR: tesseract is not installed or it's not in your PA

KeyboardInterrupt: 

In [7]:
import random
print(len(documents))
id = random.randint(0, len(documents))
print('id:',id)
print('-'*100)
print(documents[id].page_content)

17469
id: 8716
----------------------------------------------------------------------------------------------------
This, combined with a vibr ant deep-t ech ecosy st em and a sur g e in quantum startups, cr e at es a f ertil e gr ound f or inno v ation. The r obust ecosy st em of 125000 pl us startups and wide net work of MSMEs acr oss the nation will not onl y driv e domestic demand, e xport s and economic gr ow th but also position India as a l e ader in tack ling chall eng es acr oss div erse ind ustries who ar e g e aring up f or Quantum Computing now and soon f or web 3.0.


In [8]:
documents[id].metadata

{'source': '../data/TRAIN/44.pdf',
 'filename': '44.pdf',
 'extension': '.pdf',
 'relative_path': '44.pdf',
 'created_date': '2024-11-08T19:37:26.704175',
 'modified_date': '2024-07-24T16:02:18'}

---
# Test Chuncks

In [9]:
# данные для теститрования: Вопрос, Фрагмент релевантного чанка (подстрока)
tests = [
    {
        "question": "Сколько человек пользуются интернетом?",
        "answer": "По итогам 2023 года 5.3 млрд человек или 65,7% мирового населения пользовались интернетом",
        "filename": 1,
        "slide_number": 2
    },
    {
        "question": "Какая ежедневная аудитория Рунета",
        "answer": "Ежедневная аудитория Рунета достигла 95,3 млн человек, что составляет 66,4% от всего населения России",
        "filename": 1,
        "slide_number": 2
    },
    {
        "question": "Какая доля проникновения социальных сетей",
        "answer": "Проникновение социальных сетей в мире уже достигло 90%",
        "filename": 1,
        "slide_number": 2
    },
    {
        "question": "Сколько интернет пользователей смотрят видеоролики",
        "answer": "Видеоролики остаются крайне популярным форматом, практически все интернет-пользователи (92,3%) смотрят их на различных ресурсах",
        "filename": 1,
        "slide_number": 3
    },  
    {
        "question": "Какая самая крупная социальная сеть в мире",
        "answer": "Facebook с аудиторией 3030 млн",
        "filename": 1,
        "slide_number": 5
    },  
    {
        "question": "Сколько минут в день проводят пользователей в социальных сетей",
        "answer": "Среднесуточное использование социальных сетей интернет-пользователями во всем мире достигло 151 минуты в день в 2023 году",
        "filename": 1,
        "slide_number": 6
    },  
    {
        "question": "В какой стране больше всего используют социальные сети в качестве источника новостей",
        "answer": "Нигерия, 78%",
        "filename": 1,
        "slide_number": 6
    }, 
    {
        "question": "Какие основные преимущества использования социальных сетей",
        "answer": "Уровень вовлечения 86%",
        "filename": 1,
        "slide_number": 7
    }, 
    {
        "question": "Ежемесячная аудитория TikTok по странам",
        "answer": "Уровень вовлечения 86%",
        "filename": 1,
        "slide_number": 7
    }, 
        {
        "question": "What is the overall trend in playtime for gamers in 2024?",
        "answer": "Playtime has shrunk significantly, with a decrease of 26% since its peak in Q1 2021. This trend is expected to continue into 2024 as players engage less frequently with games",
        "filename": 3,
        "slide_number": 4
    }, 
    {
        "question": "How did the PC and console gaming market perform in 2023?",
        "answer": "The PC and console gaming market generated $93.5 billion in 2023, marking a modest growth of 2.6% year-over-year. However, the overall market is slowing down compared to previous years",
        "filename": 3,
        "slide_number": 8
    }, 
    {
        "question": "Which types of games are capturing most of the playtime?",
        "answer": "Games that have been available for six or more years are capturing over half of the total playtime, indicating a trend where older titles dominate player engagement",
        "filename": 3,
        "slide_number": 7
    }, 
    {
        "question": "What are the main revenue sources for PC and console gaming?",
        "answer": "In 2023, premium transactions accounted for 56% of PC spending and 57% of console spending, highlighting the importance of full-price game sales alongside in-game purchases",
        "filename": 3,
        "slide_number": 9
    },
    {
        "question": "What is the forecast for player growth in the PC and console market from 2023 to 2026?",
        "answer": "The forecast anticipates a compound annual growth rate (CAGR) of 1.6% for PC players and 3% for console players, indicating a slowdown in new player acquisition compared to previous years",
        "filename": 3,
        "slide_number": 11
    }, 
    {
        "question": "Какой онлайн кинотеатр самый популярный в России",
        "answer": "Иви",
        "filename": 13,
        "slide_number": 14
    }, 
    {
        "question": "В какой индустрии самая дорогая цена за 1000 показов",
        "answer": "Маркетинг 8.1€",
        "filename": 17,
        "slide_number": 3
    },

    {
        "question": "Какой прогноз объема российской интернет-экономики на 2023 год",
        "answer": "16.4 трлн руб.",
        "filename": 17,
        "slide_number": 3
    }, 
    {
        "question": "Сможет ли ИИ заменить человека?",
        "answer": "Людей заменить ИИ не сможет (пока), но существенно изменит суть работы человека",
        "filename": 0,
        "slide_number": 15
    },
    {
        "question": "Какова польза инфлюенс-маркетинга в сфере недвижимости?",
        "answer": "В сфере недвижимости инфлюенс-маркетинг позволяет уникально и эффективно продвигать объекты, используя данные о потребительских предпочтениях, которые предоставляют инфлюенсеры",
        "filename": 4,
        "slide_number": 30
    },
    {
        "question": "Why AI tools can be gamechangers for marketers?",
        "answer": "AI tools can be gamechangers for marketers. They can expedite the brainstorming process with data-driven suggestions, optimize content for specific channels, and help automate content repurposing across various platforms.",
        "filename": 2,
        "slide_number": 20
    },
    {
        "question": "Какая доля населения России использует отечественные социальные сети/онлайн-платформы?",
        "answer": "70% населения России пользуются отечественными социальными сетями/онлайн-платформами",
        "filename": 6,
        "slide_number": 10
    },
    {
        "question": "What are big data revolutionizing in terms of companies?",
        "answer": "Big data is revolutionizing how companies attain greater customer responsiveness and gain greater customer insights.",
        "filename": 8,
        "slide_number": 36
    },
    {
        "question": "Что представляет собой AiData GPT?",
        "answer": "AiData GPT - сегменты обезличенных идентификаторов пользователей, объединенные с помощью AI-технологии по определенным признакам",
        "filename": 10,
        "slide_number": 15
    },
    {
        "question": "Что обеспечит больший интерес клиентов к спецпроекту?",
        "answer": "Совмещение спецпроекта с распродажей обеспечит больший интерес клиентов",
        "filename": 12,
        "slide_number": 24
    },
    {
        "question": "What brands will be successful?",
        "answer": "Successful brands will be those which get clear on the role they have to play in the culture landscape, and the engagement levers needed to activate accordingly.",
        "filename": 16,
        "slide_number": 10
    },
    {
        "question": "What is social media company X planning to do to become a super app?",
        "answer": "Social media company X is equally evolving its offerings by adding video, Gen AI and finance capabilities with the aim of becoming a super app",
        "filename": 18,
        "slide_number": 11
    },
    {
        "question": "Какой результат замера по Weborama для видимости по OLV-размещениям?",
        "answer": "82% средний показатель видимости по OLV-размещениям",
        "filename": 20,
        "slide_number": 16
    },
    {
        "question": "Какая самая популярная площадка в России по объёму публикуемого контента",
        "answer": "Telegram, Dau 50.9 млн пользователей",
        "filename": 27,
        "slide_number": 3
    },
    {
        "question": "Какой самый главный тренд в видеоформатах",
        "answer": "Главный тренд — рост востребованности видеоформатов, в первую очередь коротких вертикальных роликов. Почти половина опрошенных смотрят их чаще, чем раньше, — это формат контента, популярность которого только растёт.",
        "filename": 29,
        "slide_number": 4
    },
    {
        "question": "Какой среднее время просмотра VK Видео на SmartTV",
        "answer": "125 минут в день",
        "filename": 29,
        "slide_number": 4
    }
]

In [10]:
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the model and device
# model_name = 'deepvk/USER-bge-m3'                  # +++
# model_name = 'DeepPavlov/rubert-base-cased'
model_name = 'intfloat/multilingual-e5-large'
# model_name = 'cointegrated/rubert-tiny2'
# model_name = 'intfloat/e5-mistral-7b-instruct'
# model_name = 'ai-forever/sbert_large_nlu_ru'            


embeddings_dir = '../embeddings'
model_kwargs = {'device': 'cuda'}

# Create the directory if it doesn't exist
os.makedirs(embeddings_dir, exist_ok=True)

# Load the model and tokenizer
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save the model and tokenizer to the embeddings directory
model.save_pretrained(embeddings_dir)
tokenizer.save_pretrained(embeddings_dir)

# Create embeddings using the saved model
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


In [11]:
# Transform chunks into documents
# documents = [Document(page_content=chunk.page_content if isinstance(chunk, Document) else chunk) for chunk in documents]

# Create a vector store from documents
db = FAISS.from_documents(documents, embeddings)
# Save the FAISS database locally in the embeddings directory
db.save_local(os.path.join(embeddings_dir, "faiss_db"))

# Load the model and tokenizer from the embeddings directory
loaded_model = AutoModel.from_pretrained(embeddings_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(embeddings_dir)

# Create embeddings again using the loaded model
embeddings = HuggingFaceEmbeddings(
    model_name=embeddings_dir,
    model_kwargs=model_kwargs
)

# Load the FAISS database from the embeddings directory
db = FAISS.load_local(
    os.path.join(embeddings_dir, "faiss_db"), 
    embeddings, 
    allow_dangerous_deserialization=True
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: ../embeddings


In [12]:
# Retrieve more documents with higher diversity
# Useful if your dataset has many similar documents
faiss_retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'lambda_mult': 0.25}
)

In [ ]:
import random

# Randomly select one test case
selected_test = random.choice(tests)
# Assign the topic to text_input and the chunk to answer
query = selected_test['question']
answer = selected_test['answer']

# Output the result
print(Fore.GREEN ,f"question: {query}" , Fore.RESET)
print(f"answer: {answer}")
print('='*100)

# Извлечение топ-3 кандидатов
top_documents = faiss_retriever.invoke(query)
# Печать топ-3 документов
for idx, doc in enumerate(top_documents, start=1):
    print(f"Документ {idx}:")
    print(doc.page_content)
    print("-" * 50)

In [14]:
mlmrtnrt

NameError: name 'mlmrtnrt' is not defined

---
# Self Metricks

In [ ]:
from typing import List
import torch
from transformers import AutoModel, AutoTokenizer
import faiss
from difflib import SequenceMatcher

def rag_faiss_evaluation(query: str, answer_model: List[str], model_name: str ,debug: bool = False) -> float:
    """
    Выполняет поиск и оценку с учетом точного вхождения строки запроса.
    
    Параметры:
    query (str): Исходный запрос
    answer_model (List[str]): Список строк для поиска
    debug (bool): Флаг для включения отладочной информации
    """
    def normalize_text(text: str) -> str:
        """Нормализует текст для сравнения, удаляя лишние пробелы и приводя к нижнему регистру"""
        text = text.replace('.',' .').replace(',',' ,')
        return ' '.join(text.lower().split())

    normalized_query = normalize_text(query)
    
    if debug:
        print(f"\nНормализованный запрос: {normalized_query}")

    # Проверяем сначала точное вхождение и высокое совпадение слов
    for idx, doc in enumerate(answer_model):
        normalized_doc = normalize_text(doc)
        
        if debug:
            print(f"\nДокумент {idx}:")
            print(f"Нормализованный документ: {normalized_doc}")
            print(f"Является подстрокой: {normalized_query in normalized_doc}")
            similarity = SequenceMatcher(None, normalized_query, normalized_doc).ratio()
            print(f"Процент совпадения: {similarity:.2f}")

        # Проверяем точное вхождение
        if normalized_query in normalized_doc:
            position_score = {0: 1.0, 1: 0.8, 2: 0.6, 3: 0.4, 4: 0.2}.get(idx, 0.0)
            if debug:
                print(f"Найдено точное вхождение в документе {idx}")
                print(f"Возвращаем score: {position_score}")
            return position_score

        # Проверяем покрытие слов
        words_query = set(normalized_query.split())
        words_doc = set(normalized_doc.split())
        common_words = words_query.intersection(words_doc)
        coverage = len(common_words) / len(words_query)
        
        if coverage > 0.9:
            position_score = {0: 1.0, 1: 0.8, 2: 0.6, 3: 0.4, 4: 0.2}.get(idx, 0.0)
            if debug:
                print(f"Найдено высокое совпадение в документе {idx}")
                print(f"Покрытие слов: {coverage:.2f}")
                print(f"Возвращаем score: {position_score * coverage}")
            return position_score * coverage

    # Если точного вхождения нет, используем семантический поиск
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    query_emb = model(**tokenizer(query, return_tensors="pt")).pooler_output[0]
    answer_emb = torch.stack([model(**tokenizer(a, return_tensors="pt")).pooler_output[0] for a in answer_model])
    
    index = faiss.IndexFlatL2(answer_emb.shape[1])
    index.add(answer_emb.detach().numpy())
    
    distances, indices = index.search(query_emb.detach().numpy().reshape(1, -1), 5)
    
    best_score = 0.0
    for position, idx in enumerate(indices[0]):
        text_similarity = SequenceMatcher(None, 
                                        normalize_text(query), 
                                        normalize_text(answer_model[idx])).ratio()
        position_score = {0: 1.0, 1: 0.8, 2: 0.6, 3: 0.4, 4: 0.2}.get(position, 0.0)
        score = position_score * text_similarity
        best_score = max(best_score, score)
        
        if debug:
            print(f"\nПозиция {position}, документ {idx}:")
            print(f"Текстовое сходство: {text_similarity:.2f}")
            print(f"Position score: {position_score:.2f}")
            print(f"Итоговый score: {score:.2f}")
    
    return best_score

---
# Тест: Embeddings, Similarity search
https://huggingface.co/spaces/mteb/leaderboard

In [ ]:
# Список моделей для embeddings
model_names=[
             'intfloat/e5-mistral-7b-instruct',
             'deepvk/USER-base',
             'BAAI/bge-m3',
             'intfloat/multilingual-e5-large',
             'intfloat/multilingual-e5-large-instruct',
             'deepvk/USER-bge-m3',
             'intfloat/multilingual-e5-base',
             'intfloat/multilingual-e5-small',
             'ai-forever/ru-en-RoSBERTa',
             'sergeyzh/LaBSE-ru-turbo',
             'HIT-TMG/KaLM-embedding-multilingual-mini-v1',
             'HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1',
             'sergeyzh/rubert-tiny-turbo',
             'cointegrated/rubert-tiny2',
             'ai-forever/sbert_large_mt_nlu_ru',
             'ai-forever/sbert_large_nlu_ru',
             'DeepPavlov/rubert-base-cased-sentence',
             'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
             'symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli',
            'ai-forever/ruRoberta-large',
            'ai-forever/ruT5-large',
            'ai-forever/mGPT',
            'ai-forever/FRED-T5-large-spell'
             ]

# model_kwargs = {'device': 'cpu'}
model_kwargs = {'device': 'cuda'}

# Список для хранения результатов
results = []

# Итерация по моделям
for model_name in model_names:
    print(f"Evaluating model: {model_name}")

    # Создание embeddings с использованием сохраненной модели
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs
    )
    # Create a vector store from documents
    db = FAISS.from_documents(documents, embeddings)

    # Получаем retriever с параметрами MMR
    faiss_retriever = db.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 5, 'lambda_mult': 0.25}
    )

    # Список для хранения результатов по каждой модели
    model_scores = []

    # Итерация по тестам
    for ind in range(len(tests)):
        query = tests[ind]['topic']
        right_answer = tests[ind]['chunk']

        # Извлечение топ-5 кандидатов
        top_documents = faiss_retriever.invoke(query)
        answer_model = [
            top_documents[0].page_content,
            top_documents[1].page_content,
            top_documents[2].page_content,
            top_documents[3].page_content,
            top_documents[4].page_content
        ]

        # Вычисление оценки
        score = rag_faiss_evaluation(right_answer, answer_model, model_name)
        model_scores.append(score)

    # Сохранение результатов по модели
    results.append({
        'model': model_name,
        'average_score': sum(model_scores) / len(model_scores)  # Средняя оценка по всем тестам
    })

In [ ]:
# Создание датафрейма с результатами
df_results = pd.DataFrame(results)

# Сортировка моделей по средней оценке в убывающем порядке
df_results_sorted = df_results.sort_values(by='average_score', ascending=False)

# Сохранение в файл
df_results_sorted.to_csv('model_scores.csv', index=False)

# Вывод топ-3 лучших моделей
top_models = df_results_sorted
print("Top models based on average score:")
top_models

---
# BM25

In [ ]:
# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 5  # Retrieve top results

# Список для хранения результатов
results = []

for ind in range(len(tests)):
    query = tests[ind]['topic']
    right_answer = tests[ind]['chunk']

    # Извлечение топ-5 кандидатов
    top_documents = bm25_retriever.invoke(query)
    answer_model = [
        top_documents[0].page_content,
        top_documents[1].page_content,
        top_documents[2].page_content,
        top_documents[3].page_content,
        top_documents[4].page_content
    ]

    # Вычисление оценки
    score = rag_faiss_evaluation(right_answer, answer_model)
    model_scores.append(score)

# Сохранение результатов по модели
results.append({
    'model': 'bm25_retriever',
    'average_score': sum(model_scores) / len(model_scores)  # Средняя оценка по всем тестам
})

# Создание датафрейма с результатами
df_results = pd.DataFrame(results)

# Сортировка моделей по средней оценке в убывающем порядке
df_results_sorted = df_results.sort_values(by='average_score', ascending=False)

# Сохранение в файл
df_results_sorted.to_csv('model_scores.csv', index=False)

# Вывод топ-3 лучших моделей
top_models = df_results_sorted
print("Top models based on average score:")
top_models

---
# Implementing Hybrid Search with ensemble Retrieval

In [ ]:
model_name = 'intfloat/multilingual-e5-large'
model_kwargs = {'device': 'gpu'}
# Создание embeddings с использованием сохраненной модели
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)
# Create a vector store from documents
db = FAISS.from_documents(documents, embeddings)

# Получаем retriever с параметрами MMR
faiss_retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'lambda_mult': 0.25}
)

bm25_retriever = BM25Retriever.from_documents(documents)
ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever,
                                                   bm25_retriever], weights=[0.6, 0.4 ])

# Список для хранения результатов
results = []

for ind in range(len(tests)):
    query = tests[ind]['topic']
    right_answer = tests[ind]['chunk']

    # Извлечение топ-5 кандидатов
    top_documents = ensemble_retriever.invoke(query)
    answer_model = [
        top_documents[0].page_content,
        top_documents[1].page_content,
        top_documents[2].page_content,
        top_documents[3].page_content,
        top_documents[4].page_content
    ]

    # Вычисление оценки
    score = rag_faiss_evaluation(right_answer, answer_model)
    model_scores.append(score)

# Сохранение результатов по модели
results.append({
    'model': 'EnsembleRetriever',
    'average_score': sum(model_scores) / len(model_scores)  # Средняя оценка по всем тестам
})

# Создание датафрейма с результатами
df_results = pd.DataFrame(results)

# Сортировка моделей по средней оценке в убывающем порядке
df_results_sorted = df_results.sort_values(by='average_score', ascending=False)

# Сохранение в файл
df_results_sorted.to_csv('model_scores.csv', index=False)

# Вывод топ-3 лучших моделей
top_models = df_results_sorted
print("Top models based on average score:")
top_models

---
# ReRanker
https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('amberoad/bert-multilingual-passage-reranking-msmarco',device='gpu')

# Список для хранения результатов
results = []

for ind in range(len(tests)):
    query = tests[ind]['topic']
    right_answer = tests[ind]['chunk']

    # Векторизация запроса
    query_embedding = model.encode(query)
    # Векторизация документов
    doc_embeddings = model.encode([doc.page_content for doc in documents])
    # Вычисление косинусного сходства
    cosine_scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)
    # Преобразование в список
    cosine_scores = cosine_scores[0].tolist()
    # Объединение документов и их оценок
    docs_with_scores = list(zip(documents, cosine_scores))
    # Сортировка по оценкам
    # Sort by the last float in each tuple (descending)
    top_documents = sorted(docs_with_scores, key=lambda doc: doc[1], reverse=True)
    answer_model = [
        top_documents[0][0].page_content,
        top_documents[1][0].page_content,
        top_documents[2][0].page_content,
        top_documents[3][0].page_content,
        top_documents[4][0].page_content
    ]

    # Вычисление оценки
    score = rag_faiss_evaluation(right_answer, answer_model)
    model_scores.append(score)

# Сохранение результатов по модели
results.append({
    'model': 'reranking-msmarco',
    'average_score': sum(model_scores) / len(model_scores)  # Средняя оценка по всем тестам
})

# Создание датафрейма с результатами
df_results = pd.DataFrame(results)

# Сортировка моделей по средней оценке в убывающем порядке
df_results_sorted = df_results.sort_values(by='average_score', ascending=False)

# Сохранение в файл
df_results_sorted.to_csv('model_scores.csv', index=False)

# Вывод топ-3 лучших моделей
top_models = df_results_sorted
print("Top models based on average score:")
top_models

---
# CrossEncoder
https://huggingface.co/spaces/mteb/leaderboard

In [ ]:
from typing import List, Tuple, Dict, Optional
from sentence_transformers import CrossEncoder


# cross_encoder = CrossEncoder('intfloat/e5-mistral-7b-instruct',device="cpu")
cross_encoder = CrossEncoder('BAAI/bge-m3',device="cpu")

def rerank_documents(query: str, 
                     documents: List[Document], 
                     cross_encoder: CrossEncoder, 
                     top_k: int = 5) -> List[Tuple[Document, float]]:
    """
    Переранжирование документов с использованием CrossEncoder.
    
    Args:
        query (str): Поисковый запрос
        documents (List[Document]): Список документов для ранжирования
        cross_encoder (CrossEncoder): Модель CrossEncoder
        top_k (int): Количество топовых документов для возврата
        
    Returns:
        List[Tuple[Document, float]]: Список кортежей (документ, score) отсортированный по релевантности
    """
    # Создаем пары запрос-документ для каждого документа
    pairs = [[query, doc.page_content] for doc in documents]
    
    # Получаем scores от CrossEncoder
    scores = cross_encoder.predict(pairs)
    
    # Создаем список кортежей (документ, score)
    doc_score_pairs = list(zip(documents, scores))
    
    # Сортируем по score по убыванию
    ranked_docs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)
    
    # Возвращаем top_k документов
    return ranked_docs[:top_k]

In [ ]:
# Список для хранения результатов
results = []

for ind in range(len(tests)):
    query = tests[ind]['topic']
    right_answer = tests[ind]['chunk']

    ranked_results = rerank_documents(query, documents, cross_encoder)
    answer_model = [
        top_documents[0][0].page_content,
        top_documents[1][0].page_content,
        top_documents[2][0].page_content,
        top_documents[3][0].page_content,
        top_documents[4][0].page_content
    ]

    # Вычисление оценки
    score = rag_faiss_evaluation(right_answer, answer_model)
    model_scores.append(score)

# Сохранение результатов по модели
results.append({
    'model': 'reranking-msmarco',
    'average_score': sum(model_scores) / len(model_scores)  # Средняя оценка по всем тестам
})

# Создание датафрейма с результатами
df_results = pd.DataFrame(results)

# Сортировка моделей по средней оценке в убывающем порядке
df_results_sorted = df_results.sort_values(by='average_score', ascending=False)

# Сохранение в файл
df_results_sorted.to_csv('model_scores.csv', index=False)

# Вывод топ-3 лучших моделей
top_models = df_results_sorted
print("Top models based on average score:")
top_models

---
# HybridReranker

In [ ]:
from typing import List, Tuple, Dict
from sentence_transformers import SentenceTransformer
import numpy as np
from scipy.spatial.distance import cosine
from langchain.schema import Document

class HybridReranker:
    def __init__(
        self,
        bm25_retriever,
        faiss_retriever,
        model_name='sentence-transformers/all-MiniLM-L6-v2'
    ):
        self.bm25_retriever = bm25_retriever
        self.faiss_retriever = faiss_retriever
        self.encoder = SentenceTransformer(model_name , device='cpu')
    
    def get_combined_documents(
        self,
        query: str,
        k: int = 5,
        bm25_weight: float = 0.4,
        faiss_weight: float = 0.4,
        diversity_weight: float = 0.2
    ) -> List[Document]:
        """
        Получает и комбинирует результаты из обоих ретриверов с учетом разнообразия
        
        Args:
            query: Пользовательский запрос
            k: Количество документов для возврата
            bm25_weight: Вес для BM25 скоров
            faiss_weight: Вес для FAISS скоров
            diversity_weight: Вес для разнообразия документов
        """
        # Получаем документы из обоих ретриверов
        bm25_docs = self.bm25_retriever.get_relevant_documents(query)
        faiss_docs = self.faiss_retriever.get_relevant_documents(query)
        
        # Объединяем уникальные документы
        all_docs = list({doc.page_content: doc for doc in bm25_docs + faiss_docs}.values())
        
        if not all_docs:
            return []
        
        # Получаем эмбеддинги для запроса и документов
        query_embedding = self.encoder.encode(query)
        doc_embeddings = self.encoder.encode([doc.page_content for doc in all_docs])
        
        # Рассчитываем скоры
        scores = self._calculate_scores(
            query,
            query_embedding,
            all_docs,
            doc_embeddings,
            bm25_docs,
            faiss_docs,
            bm25_weight,
            faiss_weight,
            diversity_weight
        )
        
        # Сортируем документы по финальному скору
        ranked_pairs = sorted(
            zip(all_docs, scores),
            key=lambda x: x[1],
            reverse=True
        )
        
        return [doc for doc, _ in ranked_pairs[:k]]
    
    def _calculate_scores(
        self,
        query: str,
        query_embedding: np.ndarray,
        all_docs: List[Document],
        doc_embeddings: np.ndarray,
        bm25_docs: List[Document],
        faiss_docs: List[Document],
        bm25_weight: float,
        faiss_weight: float,
        diversity_weight: float
    ) -> np.ndarray:
        """Рассчитывает комбинированные скоры для документов"""
        
        # 1. BM25 скоры
        bm25_scores = np.zeros(len(all_docs))
        for i, doc in enumerate(all_docs):
            # Если документ был в топе BM25, даем ему высокий скор
            if doc in bm25_docs:
                bm25_scores[i] = 1.0 - (bm25_docs.index(doc) / len(bm25_docs))
        
        # 2. FAISS скоры
        faiss_scores = np.zeros(len(all_docs))
        for i, doc in enumerate(all_docs):
            if doc in faiss_docs:
                faiss_scores[i] = 1.0 - (faiss_docs.index(doc) / len(faiss_docs))
        
        # 3. Разнообразие (diversity) через косинусное расстояние между документами
        diversity_scores = np.zeros(len(all_docs))
        for i, emb_i in enumerate(doc_embeddings):
            # Среднее расстояние до других документов
            distances = [
                cosine(emb_i, emb_j) 
                for j, emb_j in enumerate(doc_embeddings) 
                if i != j
            ]
            if distances:
                diversity_scores[i] = np.mean(distances)
        
        # Нормализация скоров
        bm25_scores = bm25_scores / (np.max(bm25_scores) + 1e-8)
        faiss_scores = faiss_scores / (np.max(faiss_scores) + 1e-8)
        diversity_scores = diversity_scores / (np.max(diversity_scores) + 1e-8)
        
        # Комбинируем скоры с весами
        final_scores = (
            bm25_weight * bm25_scores +
            faiss_weight * faiss_scores +
            diversity_weight * diversity_scores
        )
        
        return final_scores
    
    def analyze_ranking(
        self,
        query: str,
        documents: List[Document],
        top_k: int = 10
    ) -> List[Dict]:
        """Анализирует причины ранжирования для топ-K документов"""
        results = []
        
        query_embedding = self.encoder.encode(query)
        doc_embeddings = self.encoder.encode([doc.page_content for doc in documents])
        
        for i, doc in enumerate(documents[:top_k]):
            analysis = {
                'content': doc.page_content[:200] + "...",
                'in_bm25_top': doc in self.bm25_retriever.get_relevant_documents(query),
                'in_faiss_top': doc in self.faiss_retriever.get_relevant_documents(query),
                'semantic_similarity': 1 - cosine(query_embedding, doc_embeddings[i]),
                'metadata': doc.metadata if hasattr(doc, 'metadata') else {}
            }
            results.append(analysis)
            
        return results

# Пример использования:
def demo_reranking(query: str, bm25_retriever, faiss_retriever):
    # Создаем ранжировщик
    reranker = HybridReranker(bm25_retriever, faiss_retriever)
    
    # Получаем переранжированные документы
    reranked_docs = reranker.get_combined_documents(
        query,
        k=10,  # топ-3 документа
        bm25_weight=0.4,
        faiss_weight=0.4,
        diversity_weight=0.2
    )
    
    # Анализируем результаты
    analysis = reranker.analyze_ranking(query, reranked_docs)
    
    return reranked_docs, analysis

In [ ]:
model_name = 'intfloat/multilingual-e5-large'
model_kwargs = {'device': 'cpu'}
# Создание embeddings с использованием сохраненной модели
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)
# Create a vector store from documents
db = FAISS.from_documents(documents, embeddings)

# Получаем retriever с параметрами MMR
faiss_retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'lambda_mult': 0.25}
)
# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 5  # Retrieve top results

# Создаем ранжировщик
reranker = HybridReranker(bm25_retriever, faiss_retriever)

In [ ]:
import optuna

def objective(trial):
    bm25_weight = trial.suggest_float('bm25_weight', 0.0, 1.0)
    faiss_weight = trial.suggest_float('faiss_weight', 0.0, 1.0)
    diversity_weight = 1 - bm25_weight - faiss_weight
    
    model_scores = []
    for ind in range(len(tests)):
        query = tests[ind]['topic']
        right_answer = tests[ind]['chunk']
        
        reranked_docs = reranker.get_combined_documents(
            query,
            k=5,
            bm25_weight=bm25_weight,
            faiss_weight=faiss_weight,
            diversity_weight=diversity_weight
        )
        answer_model = [
            reranked_docs[0].page_content,
            reranked_docs[0].page_content,
            reranked_docs[0].page_content,
            reranked_docs[0].page_content,
            reranked_docs[0].page_content
        ]
        
        score = rag_faiss_evaluation(right_answer, answer_model)
        model_scores.append(score)
    
    average_score = sum(model_scores) / len(model_scores)
    return average_score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print('Optimal hyperparameters:')
print(f'bm25_weight: {study.best_params["bm25_weight"]}')
print(f'faiss_weight: {study.best_params["faiss_weight"]}')
print(f'diversity_weight: {1 - study.best_params["bm25_weight"] - study.best_params["faiss_weight"]}')
print(f'Best average score: {study.best_value}')